In [ ]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch.optim import Adam
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Resize((224, 224)),
    transforms.Normalize(mean=0.1307, std=0.3081)

])

In [ ]:
#加入了一个dropout层的Inception块
class Inception(nn.Module):
    def __init__(self,in_channels,c1,c2,c3,c4):
        super(Inception,self).__init__()
        self.p1 = nn.Conv2d(in_channels = in_channels,out_channels = c1,kernel_size = 1)
        self.p2 = nn.Sequential(
            nn.Conv2d(in_channels = in_channels,out_channels = c2[0],kernel_size = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = c2[0],out_channels = c2[1],kernel_size = 3,padding = 1),
            nn.ReLU()
        )
        self.p3 = nn.Sequential(
            nn.Conv2d(in_channels = in_channels,out_channels = c3[0],kernel_size = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = c3[0],out_channels = c3[1],kernel_size = 5,padding = 2),
            nn.ReLU()
        )
        self.p4 = nn.Sequential(
            nn.MaxPool2d(kernel_size = 3,padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = in_channels,out_channels = c4,kernel_size = 1),
            nn.ReLU()
        )
        self.dp = nn.Dropout(0.3)

    def forward(self,x):
        p1 = self.dp(self.p1(x))
        p2 = self.dp(self.p2(x))
        p3 = self.dp(self.p3(x))
        p4 = self.dp(self.p4(x))
        return torch.cat((p1,p2,p3,p4),1)

